#Long Short-Term Memory for Sequential Predictions

Gage DeZoort\
3/25/24\
\
Based on several helpful tutorials:\
[1] [LSTM for Time Series Prediction in PyTorch](https://machinelearningmastery.com/lstm-for-time-series-prediction-in-pytorch/)\
[2] [Predicting airline passengers using LSTM and Tensorflow](https://matthewmacfarquhar.medium.com/predicting-airline-passengers-using-lstm-and-tensorflow-ab86347cf318)


## Temperature Predictions

This project is based on Long Short-Term Memory (LSTM) modules. LSTMs belong to the class of Recurent Neural Networks (RNNs), which operate on sequential data (ordered data, indexed by time or space). For example, the daily temperature is a time series we all experience:

![Weather](https://www.influxdata.com/wp-content/uploads/time-series-data-weather-data.png "weather")
(Image from [this article](https://www.influxdata.com/what-is-time-series-data/))

Sentences are another example of sequential data:

![Sentences](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*phpgEszN4Q6n_Rtd24zpGw.png "sentences")
(Image from [this article](https://bansalh944.medium.com/text-generation-using-lstm-b6ced8629b03))

We see that sequential data is everywhere! RNNs have accordingly bene applied to a wide variety of domains, including:

- Natural Language Processing (NLP): translation, word prediction, sentiment analysis
- Time-Series Analysis: financial predictions, weather/climate forecasting
- Music Generation: e.g. composition
- Robotics: e.g. path predictions

How do RNNs work? Here's a helpful diagram:

![RNN](https://colah.github.io/posts/2015-08-Understanding-LSTMs/img/RNN-unrolled.png "rnn")
(Image from [this article](https://colah.github.io/posts/2015-08-Understanding-LSTMs/))

In this diagram, $A$ represents the NN. Here, we take $x_t$ to represent the sequence of inputs ($x_0$, $x_1$, $x_2$,...,$x_N$), and $h_t$ its sequence of outputs ($h_0$, $h_1$, $h_2$,...,$h_N$). The sequential nature of the predictions is highlight by the rightward arrows; the prediction at each timestep is informed by the prediction at the previous timestep. Unfortunately, it has been shown that simple RNNs *fail to learn long-term dependencies*. This was the motivation for developing LSTMs.

Okay, let's switch to a bit of coding.


In [1]:
import torch
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

# grab data
!wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv

KeyboardInterrupt: 

## Dataset Preparation

In the last line, we grabed a CSV (comma-separated value) file called `airline-passengers.csv`. Let's use Pandas to explore the data.

In [ ]:
df = pd.read_csv("airline-passengers.csv")
df.head()

We see that this is a time-series, counting the number of passengers (in units of 1,000) between Jan. 1949 and Dec. 1960, corresponding to 12 years and 144 observations. Let's plot the trend:

In [ ]:
plt.plot(df.Passengers)
plt.xlabel("Months Since 01/1949")
plt.ylabel("Airline Passengers / 1,000")
plt.show()

What trends do you observe?

The upward trend may be difficult for the ML model to capture given the limited size of the dataset. We can simply remove it before training the algorithm, then add it back! Our goal will be to fit a quadratic to the data:

`P(m) = x_0 + x_1 * m + x_2 * m^2`

Where `P(m)` is the number of passengers in a given month `m`. Let's grab the regression coefficients:

In [ ]:
N = len(df)
ones, xrange = np.ones(N), np.arange(N)
X = np.stack((ones, xrange, xrange**2)).T
y = df.Passengers.to_numpy().reshape(-1,1)
beta = (np.linalg.inv(X.T @ X)@X.T@y)
x0, x1, x2 = beta[0][0], beta[1][0], beta[2][0]
x0, x1, x2

Let's plot the corrected passengers data:

In [ ]:
passengers = df.Passengers
plt.plot(passengers, label="Raw Data")

trend = x0 + xrange*x1 + xrange**2 * x2
passengers_c = passengers - trend

plt.plot(trend, "r-", label="Global Trend")
plt.plot(passengers_c, "g--", label="Corrected")
plt.xlabel("Months Since 01/1949")
plt.ylabel("Airline Passengers / 1,000")
plt.legend(loc="best")
plt.show()

## Building Train/Test Sets

In [ ]:
# convert everything to plain arrays
passengers = passengers.values.astype("float32").reshape(-1,1)
passengers_c = passengers_c.values.astype("float32").reshape(-1,1)
passengers_c.shape

Let's turn this into an ML task. Given 30% of the time series, can we predict the remaining 70%? Fundamentally, that means we're doing regression.

To train a model, we need to show it data in the time interval $[t-w, t-1]$, where $w$ is the "sequence length" or "lookback" size, and ask it to make predictions for the timestep $t$. To do this, we need to turn our training data into $(X,y)$ pairs,  $X,y\in\mathbb{R}^{w}$, where $X$ reprsents the inputs and $y$ represents the targets.




In [ ]:
# function to create windows [t-w,t-1] with corresponding truth labels [t]
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

# scale the data in a nice way
sc = MinMaxScaler()
training_data = sc.fit_transform(passengers_c)

# use a lookback window of size 4
seq_length = 4
x, y = sliding_windows(training_data, seq_length)

train_size = int(len(y) * 0.30)
test_size = len(y) - train_size

data_x = torch.Tensor(np.array(x))
data_y = torch.Tensor(np.array(y))

train_x = torch.Tensor(np.array(x[0:train_size]))
train_y = torch.Tensor(np.array(y[0:train_size]))

test_x = torch.Tensor(np.array(x[train_size:len(x)]))
test_y = torch.Tensor(np.array(y[train_size:len(y)]))

print(training_data.shape)
print(data_x.shape)
print(data_y.shape)
print(train_x.shape)
print(train_y.shape)

## Building an LSTM
Now we're going to build a LSTM! Here's the PyTorch model:

In [ ]:
from torch import nn

class LSTM(nn.Module):

    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()

        self.num_classes = num_classes
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_length = seq_length

        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)

        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        # Propagate input through LSTM
        _, (h_out, _) = self.lstm(x)
        h_out = h_out.view(-1, self.hidden_size)
        out = self.fc(h_out)

        return out

This model has several components. The main workhorse is the **LSTM Module**: see the [PyTorch docs](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) for details of the implementation. Here's a diagram:

![LSTM](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*kT7TJdlJflJJSnEJ6XRKug.png "lstm")\
(Image from [this article](https://bansalh944.medium.com/text-generation-using-lstm-b6ced8629b03))
\
\
This is a single LSTM "block" corresponding to the timestep $t$. There's a lot going on here, but here's the gist:

- The LSTM block at timestep $t$ is fed by the input $x_t$ (# passengers), the output from the previous block $h_{t-1}$, and the memory from the previous block $c_{t-1}$.
- The LSTM block at timestep $t$ is composed of several logical gates. These include an input gate, a forget gate, a cell gate, and an output gate. The full system of equations is

$$
\begin{align*}
  i_t &= \sigma(W_{ii} x_t + b_{ii} + W_{hi}h_{t-1} + b_{hi})\ &\rightarrow \ \ \text{input gate} \\
  f_t &= \sigma(W_{if}x_t + b_{if} + W_{hf}h_{t-1} + b_{hf})\ &\rightarrow \ \ \text{forget gate}\\
  g_t &= \text{tanh}(W_{ig}x_t + b_{ig} + W_{hg}h_{t-1} + b_{hg})\ &\rightarrow \ \ \text{cell features}\\
  o_t &= \sigma(W_{io}x_t + b_{io} + W_{ho}h_{t-1} + b_{ho})\ &\rightarrow \ \ \text{output gate}\\
  c_t &= f_t \odot c_{t-1} + i_t \odot g_t\ &\rightarrow \ \ \text{cell state (memory)}\\
  h_t &= o_t \odot \text{tanh}(c_t) \ &\rightarrow \ \ \text{hidden state}
\end{align*}
$$

In practice, the PyTorch module `nn.LSTM()` has inputs `input_size` corresponding to the dimension of $x_t$, `hidden_size` corresponding to the size of the outputs $h_{t}$, and `num_layers` corresponding to the number of "stacked" LSTM modules. Let's train an our model:

In [ ]:
num_epochs = 2000
learning_rate = 0.01

input_size = 1
hidden_size = 16
num_layers = 1

num_classes = 1

lstm = LSTM(num_classes, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    outputs = lstm(train_x)
    optimizer.zero_grad()

    # obtain the loss function
    loss = criterion(outputs, train_y)

    loss.backward()

    optimizer.step()
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.5f" % (epoch, loss.item()))


In [ ]:
lstm.eval()

# predict on training and test data
all_predict = lstm(data_x)

data_predict = all_predict.data.numpy()
data_y_plot = data_y.data.numpy()

data_predict = sc.inverse_transform(data_predict).flatten()
data_y_plot = sc.inverse_transform(data_y_plot).flatten()

plt.axvline(x=train_size, c='r', linestyle='--', label='Train/Test Split')

plt.plot(data_y_plot + trend[:-seq_length-1], label='Ground Truth')
plt.plot(data_predict + trend[:-seq_length-1], label='predictions')
plt.suptitle('Time-Series Prediction')
plt.legend()
plt.show()